In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import matplotlib as mpl
import matplotlib.pyplot as plt
!pip install py7zr
from keras.preprocessing.image import load_img,img_to_array
from py7zr import unpack_7zarchive
import shutil
import os
shutil.register_unpack_format('7zip', ['.7z'], unpack_7zarchive)

     |███████████████                 | 30 kB 1.3 MB/s eta 0:00:01

     |████████████████████            | 40 kB 1.2 MB/s eta 0:00:01

     |████████████████████████████████| 65 kB 580 kB/s 


     |█▏                              | 71 kB 2.5 MB/s eta 0:00:01

     |█▊                              | 102 kB 1.3 MB/s eta 0:00:02

...

     |█████████████████████▌          | 1.5 MB 6.5 MB/s eta 0:00:01

     |██████████████████████████▉     | 1.8 MB 6.5 MB/s eta 0:00:01

     |███████████████████████████████ | 2.1 MB 6.5 MB/s eta 0:00:01

     |████████████████████████████████| 2.2 MB 6.5 MB/s 


     |████████████████████████████████| 121 kB 8.2 MB/s 


In [2]:
shutil.unpack_archive('/kaggle/input/cifar-10/train.7z', '/kaggle/working')
train_dir = os.listdir("./train");
train_dir_len = len(train_dir)

In [3]:
Train_Labels = pd.read_csv('/kaggle/input/cifar-10/trainLabels.csv')
Train_Images = pd.DataFrame(columns = ['id','label','path'],dtype=str)

In [ ]:
path_base = '/kaggle/working/train/'

for index in range(0,train_dir_len):
    path = path_base + str(index+1)+'.png'
    if os.path.exists(path):
        Train_Images = Train_Images.append([{ 'id': str(Train_Labels['id'].iloc[index]),'path': path, 'label':Train_Labels['label'].iloc[index]}])

In [ ]:
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
for name in  class_names:
    index = class_names.index(name)
    Train_Images.loc[Train_Images.label==name,'label'] = str(index)

In [ ]:
train_data_generator = ImageDataGenerator(
            rescale=1./255.,
            validation_split=0.2,
            horizontal_flip=True
            )

In [ ]:
train_generator = train_data_generator.flow_from_dataframe(dataframe=Train_Images,
            directory="./train/",
            x_col="path",
            y_col="label",
            subset="training",
            batch_size=100,
            shuffle=True,
            target_size=(32,32),
            class_mode="categorical")

In [ ]:
validation_generator = train_data_generator.flow_from_dataframe(dataframe=Train_Images,
            directory="./train/",
            x_col="path",
            y_col="label",
            subset="validation",
            batch_size=100,
            shuffle=True,
            target_size=(32,32),
            class_mode="categorical")

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32,  3)))

model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(64, (2, 2), activation='relu',padding='same'))
model.add(keras.layers.MaxPooling2D(1, 1))

model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(128, (3, 3), activation='relu',padding='same'))
model.add(keras.layers.MaxPooling2D(2, 2))

model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(256, (2, 2), activation='relu',padding='same'))
model.add(keras.layers.MaxPooling2D(1, 1))


model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation='elu'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(64, activation='elu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(10, activation="softmax"))

model.compile(loss=keras.losses.CategoricalCrossentropy(from_logits=True), 
 optimizer='adam',metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator, 
                    steps_per_epoch=(40000//100),
                    epochs= 20,
                    validation_data=validation_generator,
                   validation_steps=(10000//100)
                   )

In [ ]:
if os.path.exists("./test"):
    shutil.rmtree("./test")
if os.path.exists("./train"):
    shutil.rmtree("./train")
if not os.path.exists("./data"):
    os.mkdir("./data")
shutil.unpack_archive('/kaggle/input/cifar-10/test.7z', '/kaggle/working/data')
test_dir = os.listdir("./data/test");
test_dir_len = len(test_dir)

In [ ]:
test_data_generator = ImageDataGenerator(rescale=1./255.)
test_generator = test_data_generator.flow_from_directory(directory='/kaggle/working/data',
            batch_size=100,
            shuffle=False,color_mode='rgb',
            target_size=(32,32),
            class_mode=None)

In [ ]:
predict_test = model.predict_generator(test_generator)

In [ ]:
predict_generator = np.argmax(predict_test, axis=1)
predict_generator[:2],[class_names[int(i)] for i in predict_generator[:2]]

In [ ]:
submission = pd.DataFrame(columns = ['id','label'],dtype=str)
submission["label"] = [class_names[int(i)] for i in predict_generator]
submission["id"] = [ (''.join(filter(str.isdigit, name ))) for name in test_generator.filenames]

In [ ]:
submission.to_csv("submission.csv",index=False)